# Capstone Project - Battle of Neighborhoods: Optimal Location for the Opening of an Italian Restaurant

### _by Gabriella Mansur_

#### Please refer to the main report to further explanation of this code.

### Import packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Scrape dataset from Wikipedia

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami", header=0)[0]

In [3]:
df.head()

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257


### Prepare the data

In [4]:
# Drop unnecessary columns
df.drop(['Demonym', 'Sub-neighborhoods'], axis=1, inplace=True)

# Drop last row which is sum of all values
df.drop(df.tail(1).index,inplace=True)

In [5]:
# Correct the format of population number
df['Population2010'] = df['Population2010'].str.replace(r"\(.*\)","")
df["Population2010"] = df["Population2010"].str.replace(",","")

In [6]:
# Drop locations without coordinates
df.dropna(subset=['Coordinates'], inplace=True) 

In [7]:
# Separate coordinates into latitude and longitude
df["Latitude"] = df["Coordinates"].str[0:6].astype(float)
df["Longitude"] = df["Coordinates"].str[6:14].astype(float)
df.drop("Coordinates", axis=1, inplace=True)

### Generate map of Miami

In [8]:
#Getting coordinates of Miami

address = 'Miami, FL'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Miami are 25.7742658, -80.1936589.


In [26]:
# Create map of Miami using the latitude and longitude values

map_miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miami)  

In [27]:
map_miami

#### Define foursquare credentials and version

In [10]:
CLIENT_ID = 'YBTUXW5S1ORKODKGGZR5JRX12JT5F3RTVL4GKLJI3EKQJ114' # your Foursquare ID
CLIENT_SECRET = 'NU0N33Z4A452PX5M5V3GIOQGQHCCY3BJGMGZE1Q5C2SADNWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YBTUXW5S1ORKODKGGZR5JRX12JT5F3RTVL4GKLJI3EKQJ114
CLIENT_SECRET:NU0N33Z4A452PX5M5V3GIOQGQHCCY3BJGMGZE1Q5C2SADNWL


### Retrieve venues in Miami using Foursquare

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
miami_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Midtown
Overtown
Park West
The Roads
Upper Eastside
Venetian Islands
Virginia Key
West Flagler
Wynwood


In [13]:
# one hot encoding
miami_onehot = pd.get_dummies(miami_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
miami_onehot['Neighborhood'] = miami_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [miami_onehot.columns[-1]] + list(miami_onehot.columns[:-1])
miami_onehot = miami_onehot[fixed_columns]

miami_onehot.head()

,Yoga Studio,Airport,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Garden,Big Box Store,Boat or Ferry,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Event Space,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Latin American Restaurant,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Night Market,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Theater,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
miami_grouped = miami_onehot.groupby('Neighborhood').mean().reset_index()
miami_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Baseball Stadium,Basketball Stadium,Beer Garden,Big Box Store,Boat or Ferry,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Cruise,Cuban Restaurant,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Event Space,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Latin American Restaurant,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Night Market,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Print Shop,Pub,Public Art,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Bar,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Theater,Trail,Train Station,Tree,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint
0,Allapattah,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000
1,Arts & Entertainment District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

### Narrow down to Italian restaurants

In [15]:
# Making the plot of fraction of Italian restaurants in Miami
ita_rest = miami_grouped[['Neighborhood','Italian Restaurant']]
ita_rest_plot = ita_rest.sort_values(by=['Italian Restaurant'],ascending=False)
ita_rest_plot.index = ita_rest_plot['Neighborhood']
ita_rest_plot = ita_rest_plot[0:9]
ita_rest_plot.plot.barh(legend=False,figsize=(8,5))

In [16]:
num_top_venues = 3

for hood in miami_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = miami_grouped[miami_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allapattah----
                 venue  freq
0    Food & Drink Shop  0.15
1                 Park  0.15
2  American Restaurant  0.08


----Arts & Entertainment District----
              venue  freq
0    Ice Cream Shop  0.07
1  Tapas Restaurant  0.07
2              Park  0.07


----Brickell----
                 venue  freq
0                Hotel  0.17
1   Italian Restaurant  0.10
2  Japanese Restaurant  0.07


----Buena Vista----
                venue  freq
0  Italian Restaurant  0.07
1           Nightclub  0.07
2       Jewelry Store  0.07


----Coconut Grove----
           venue  freq
0           Park  0.27
1  Boat or Ferry  0.18
2          Plaza  0.09


----Coral Way----
           venue  freq
0   Tennis Court  0.08
1           Café  0.08
2  Grocery Store  0.08


----Design District----
                venue  freq
0                Café  0.10
1  Italian Restaurant  0.07
2       Jewelry Store  0.07


----Downtown----
                  venue  freq
0    Italian Restaurant  0.10
1      

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Get 3 top venues in each neighborhood

In [18]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = miami_grouped['Neighborhood']

for ind in np.arange(miami_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(miami_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allapattah,Food & Drink Shop,Park,Cosmetics Shop
1,Arts & Entertainment District,Park,Ice Cream Shop,New American Restaurant
2,Brickell,Hotel,Italian Restaurant,Argentinian Restaurant
3,Buena Vista,Italian Restaurant,Nightclub,Coffee Shop
4,Coconut Grove,Park,Boat or Ferry,Trail
5,Coral Way,Park,Grocery Store,Café
6,Design District,Café,Italian Restaurant,Nightclub
7,Downtown,Italian Restaurant,Brazilian Restaurant,Sandwich Place
8,Edgewater,Peruvian Restaurant,Restaurant,Gym / Fitness Center
9,Flagami,Bakery,Liquor Store,Seafood Restaurant


### Classify neighborhoods in two different labels - purple for those where Italian restaurants are among the top 3 common venues, or red if not

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', "")

miami_merged = df

miami_merged = miami_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how = 'right')

In [20]:
miami_merged['Cluster Labels'] = np.where((miami_merged['1st Most Common Venue'] == 'Italian Restaurant') | (miami_merged['2nd Most Common Venue'] == 'Italian Restaurant') | (miami_merged['3rd Most Common Venue'] == 'Italian Restaurant'), 1, 0)

### Create map with markers - purple if Italian restaurant is among the top 3 locations, red if not

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(0,3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(miami_merged['Latitude'], miami_merged['Longitude'], miami_merged['Neighborhood'], miami_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [22]:
miami_merged

,Neighborhood,Population2010,Population/Km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Allapattah,54289,4401,25.815,-80.224,0,Food & Drink Shop,Park,Cosmetics Shop
1,Arts & Entertainment District,11033,7948,25.799,-80.190,0,Park,Ice Cream Shop,New American Restaurant
2,Brickell,31759,14541,25.758,-80.193,1,Hotel,Italian Restaurant,Argentinian Restaurant
3,Buena Vista,9058,3540,25.813,-80.192,1,Italian Restaurant,Nightclub,Coffee Shop
4,Coconut Grove,20076,3091,25.712,-80.257,0,Park,Boat or Ferry,Trail
5,Coral Way,35062,4496,25.750,-80.283,0,Park,Grocery Store,Café
6,Design District,3573,3623,25.813,-80.193,1,Café,Italian Restaurant,Nightclub
7,Downtown,71000,10613,25.774,-80.193,1,Italian Restaurant,Brazilian Restaurant,Sandwich Place
8,Edgewater,15005,6675,25.802,-80.190,0,Peruvian Restaurant,Restaurant,Gym / Fitness Center
9,Flagami,50834,5665,25.762,-80.316,0,Bakery,Liquor Store,Seafood Restaurant
